# Web Application for an ETF Analyzer

In this project, we develop a financial database and a web application to analyze the performance of a hypothetical FinTech Exchange-Traded Fund (ETF). The application is built using SQL, Python, and the Voilà library. The ETF comprises four stocks: GDOT (Green Dot Corporation), GS (Goldman Sachs), PYPL (PayPal Holdings), and SQ (Block, formerly Square). Each stock has its own dedicated table within the etf.db database.

The analysis focuses on the daily returns of the individual ETF stocks, as well as the performance of the ETF as a whole. Once the analysis is complete, the visualizations are deployed to a web application using the Voilà library, which allows us to transform the Jupyter Notebook into an interactive web interface.

Project stages:

0. Populate ETF data data
1. Analyze a single asset in the ETF
2. Optimize data access with Advanced SQL queries
3. Analyze the ETF portfolio
4. Create a new database where to store the data of the ETF returns equally weighted.The idea is to not to alter the original.
5. Deploy the notebook as a web application


In [1]:
%load_ext autoreload
%autoreload 2

## Imports and initial settings

In [8]:
import numpy as np
import pandas as pd
import hvplot.pandas
import sqlalchemy
import datetime
from datetime import date, datetime, timedelta

from utils import populate_etf_tables, setup_logging, retrieve_yahoo_prices_and_volume

In [3]:
# Parameters 
log_level = 'INFO'
etf_tickers = ['GDOT', 'GS', 'PYPL', 'SQ'] # stocks Green Dot Inc, Goldman Scahs Group Inc, Paypal Inc, and Square Inc.

# Dates - Use format '%Y-%m-%d'
end_date = datetime.now().date() - timedelta(days=1)
start_date = end_date - timedelta(days=5*365)

start_date, end_date

(datetime.date(2019, 10, 17), datetime.date(2024, 10, 15))

### 0. Populate Analysis Data: prices and returns
Data source: Yahoo Finance

In [4]:
setup_logging(log_level)

In [5]:
test = retrieve_yahoo_prices_and_volume(start_date=start_date, end_date=end_date)
test['ticker'], test['market_data']

2024-10-16 08:44:15,567 - INFO - NumExpr defaulting to 4 threads.


('spy',
                   open        high         low       close    volume
 time                                                                
 2019-10-17  277.411898  277.930284  276.338090  277.041626  45736600
 2019-10-18  276.495419  277.152654  274.958764  275.828918  64304000
 2019-10-21  277.171229  277.902506  276.726885  277.698853  39048600
 2019-10-22  278.244943  278.541172  276.699051  276.791626  48594700
 2019-10-23  276.532413  277.652494  276.319493  277.596954  34352200
 ...                ...         ...         ...         ...       ...
 2024-10-08  570.419983  573.780029  569.530029  573.169983  37398700
 2024-10-09  573.159973  577.710022  572.549988  577.140015  37912200
 2024-10-10  575.770020  577.580017  574.489990  576.130005  44138100
 2024-10-11  576.049988  580.330017  575.909973  579.580017  42268000
 2024-10-14  581.219971  585.270020  580.729980  584.320007  36217200
 
 [1256 rows x 5 columns])

In [11]:
# Getting the data
database_connection_string = 'sqlite:///etf.db'
populate_etf_tables(etf_tickers, database_connection_string)

2024-10-16 08:53:31,224 - INFO - Retrieve of Market Data and Update of SQL-Tables Completed


['GDOT', 'GS', 'PYPL', 'SQ']

In [12]:
# Create an engine to interact with the SQLite database
engine = sqlalchemy.create_engine(database_connection_string)

In [77]:
#sqlalchemy.create_engine?

## 1. Analyze a single asset in the FinTech ETF


### Step 1.1: Load the PYPL data from the database into a Pandas DataFrame.

In [13]:
# Write a SQL query to SELECT all of the data from the PYPL table
query = """
    SELECT * from PYPL;
"""

# Use the query to read the PYPL data into a Pandas DataFrame and set index to "time"
fmt = '%Y%m%d %H:%M:%S'
pypl_dataframe = pd.read_sql_query(query, con=engine, parse_dates={'time':fmt})
pypl_dataframe = pypl_dataframe.set_index('time')

print("\033[1m  Table with Paypal Inc. prices, volume and daily returns")
pypl_dataframe

  Table with Paypal Inc. prices, volume and daily returns


,open,high,low,close,volume
time,,,,,
2019-10-17,103.889999,104.339996,102.839996,103.629997,3996300
2019-10-18,103.919998,104.320000,100.750000,101.220001,7260700
2019-10-21,102.360001,102.730003,100.464996,101.440002,6694400
2019-10-22,101.360001,101.949997,97.114998,97.360001,11298100
2019-10-23,97.419998,97.599998,94.769997,96.639999,16470200
...,...,...,...,...,...
2024-10-08,79.989998,81.400002,79.574997,81.160004,8618500
2024-10-09,80.879997,82.000000,80.849998,81.650002,6926000
2024-10-10,79.940002,80.074997,78.529999,78.980003,11326100


### Step 1.2: Review the first five and the last five rows of the DataFrame. We save the period length for future annualization.

In [21]:
# Print Beggining and End date of the Period.
start = pypl_dataframe.index[0]
end = pypl_dataframe.index[-1]
print(f"Beggining of period: {start}")
print(f"End of period      : {end}")

# Calculate lenght of period to calculate actual annualized return later
period = (pypl_dataframe.index[-1] - pypl_dataframe.index[0])
period_in_years = period.days/365.25
print(f"Period in days is: {period.days}, which are {period_in_years:.2f} years \n\n")

# View the first 5 rows of the DataFrame.
print("\033[1m  Firsts and lasts columns of the pypl_dataframe, with data of Paypal stock prices, volume, and returns. \n")
display(pypl_dataframe.head())
display(pypl_dataframe.tail())

Beggining of period: 2019-10-17 00:00:00
End of period      : 2024-10-14 00:00:00
Period in days is: 1824, which are 4.99 years 


  Firsts and lasts columns of the pypl_dataframe, with data of Paypal stock prices, volume, and returns. 



,open,high,low,close,volume
time,,,,,
2019-10-17,103.889999,104.339996,102.839996,103.629997,3996300
2019-10-18,103.919998,104.320000,100.750000,101.220001,7260700
2019-10-21,102.360001,102.730003,100.464996,101.440002,6694400
2019-10-22,101.360001,101.949997,97.114998,97.360001,11298100
2019-10-23,97.419998,97.599998,94.769997,96.639999,16470200


,open,high,low,close,volume
time,,,,,
2024-10-08,79.989998,81.400002,79.574997,81.160004,8618500
2024-10-09,80.879997,82.000000,80.849998,81.650002,6926000
2024-10-10,79.940002,80.074997,78.529999,78.980003,11326100
2024-10-11,79.540001,80.725998,79.160004,80.510002,7744800
2024-10-14,80.339996,80.980003,79.820000,80.669998,5865400


In [23]:
pypl_dataframe['daily_returns'] = pypl_dataframe['close'].pct_change()
pypl_dataframe["Daily Returns %"] = pypl_dataframe['daily_returns'] * 100

### Step 1.3: Interactive visualization for the PYPL daily returns using hvPlot.

In [7]:
# Create an interactive visualization with hvplot to plot the daily returns for PYPL.
pypl_dataframe.hvplot(
    title="PAYPAL Daily Returns (%)",
    y='Daily Returns %',
    xlabel='Date',
    ylabel='Returns (%)',
    width=800
).opts(color='blue')

:Curve   [time]   (Daily Returns %)

### Step 1.4: Interactive visualization for the PYPL cumulative returns. 

In [9]:
# Representing the cummulative investment
growth_of_1usd_investment = (1 + pypl_dataframe["daily_returns"]).cumprod()

# Transforming a series to a dataframe and renaming columns
growth_of_1usd_investment = growth_of_1usd_investment.to_frame().rename(columns={'daily_returns': 'Growth of 1 USD Investment'})

print("\033[1m Table: Evolution of a $1 initial investment on Dec 15th 2016 on the ETF.")
display(growth_of_1usd_investment)

# Create an interactive visaulization with hvplot to plot the cumulative returns for PYPL.
growth_of_1usd_investment.hvplot(
    title=f"Paypal Holdings Inc -- Growth of 1 USD Initial Investment -- Period: {start.date()} to {end.date()}",
    ylabel="Initial Investment \n plus Cumulative Return",
    xlabel="Date",
    width=900
)

 Table: Evolution of a $1 initial investment on Dec 15th 2016 on the ETF.


,Growth of 1 USD Investment
time,
2016-12-16,0.994436
2016-12-19,0.997724
2016-12-20,1.005058
2016-12-21,1.013910
2016-12-22,1.003541
...,...
2020-11-30,5.417299
2020-12-01,5.475974
2020-12-02,5.378351


:Curve   [time]   (Growth of 1 USD Investment)

## 2. Optimize the SQL Queries

For this part, we continue to analyze a single asset (PYPL) from the ETF. We use SQL queries to optimize the efficiency of accessing data from the database.



### Step 2.1: Access the closing prices for PYPL that are greater than 200 dollars

In [16]:
# Write a SQL SELECT statement to select the time column 
# where the PYPL closing price was higher than 200.0.
query = """
    SELECT time 
    FROM PYPL
    WHERE close > 200;
"""

# Using the query, read the data from the database into a Pandas DataFrame, and convert date strings to date
fmt = '%Y%m%d %H:%M:%S'
pypl_dates_higher_than_200 = pd.read_sql_query(query, engine, parse_dates={'time': fmt})

pypl_higher_than_200 = pypl_dataframe.loc[pypl_dates_higher_than_200['time'], 'close'].to_frame()
print("\n")
print("\033[1m  Older dates when Paypal close price is higher than $200 in available data")
pypl_higher_than_200.head()



  Older dates when Paypal close price is higher than $200 in available data


,close
time,
2020-08-05,202.920
2020-08-06,204.090
2020-08-25,201.710
2020-08-26,203.530
2020-08-27,204.340
2020-08-28,204.480
2020-08-31,203.950
2020-09-01,208.920
2020-09-02,210.820


### Step 2: Find the top 10 daily returns for PYPL

In [17]:
# Write a SQL SELECT statement to select the time and daily_returns columns
# Sort the results in descending order and return only the top 10 return values
query = """
    SELECT time, daily_returns
    FROM PYPL
    ORDER by daily_returns  desc
    LIMIT 10;
"""

# Using the query, read the data from the database into a Pandas DataFrame
# Counting is useful to visualize amount of data, so index is not change to time
fmt = '%Y%m%d %H:%M:%S'
pypl_top_10_returns = pd.read_sql_query(query, engine, parse_dates={'time':fmt})

pypl_top_10_returns['daily_returns'] = pypl_top_10_returns['daily_returns'] * 100
    
# Review the resulting DataFrame
print ("\n")
print("\033[1m Table with the top 10 larger daily returns of Paypal stock in percentages (%):")
display(round(pypl_top_10_returns,2))




 Table with the top 10 larger daily returns of Paypal stock in percentages (%):


,time,daily_returns
0,2020-03-24,14.10
1,2020-05-07,14.03
2,2020-03-13,13.87
3,2020-04-06,10.09
4,2018-10-19,9.34
5,2019-10-24,8.59
6,2020-11-04,8.10
7,2020-03-10,8.09
8,2020-04-22,7.53
9,2018-12-26,7.47


## 3. Analyze the Fintech ETF Portfolio

For this part, we build the entire ETF portfolio and then evaluate its performance. To do so, we build the ETF portfolio by using SQL joins to combine all the data for each asset.


### Step 3.1: Write a SQL query to join each table in the portfolio into a single DataFrame.

In [21]:
# We join all the portfolio tables on the time column
query = """
    SELECT  GS.time as index_time, *
    FROM GDOT, GS, PYPL, SQ
    WHERE GDOT.time = GS.time
        AND   PYPL.time = SQ.time
        AND   GDOT.time = PYPL.time;
"""
# Read the data from the database into a Pandas DataFrame, and set up date index
frm = '%Y%m%d %H:%M:%S'

etf_portfolio = pd.read_sql_query(query, engine, parse_dates={'index_time': frm})
etf_portfolio.set_index(etf_portfolio['index_time'].dt.date, inplace=True)
etf_portfolio.drop(columns='index_time', inplace=True)

# Review the resulting DataFrame
print('\n')
print("\033[1m Joined Tables from ['GDOT', 'GS', 'PYPL', 'SQ'] on dates")
etf_portfolio.head()



 Joined Tables from ['GDOT', 'GS', 'PYPL', 'SQ'] on dates


,time,open,high,low,close,volume,daily_returns,time,open,high,...,close,volume,daily_returns,time,open,high,low,close,volume,daily_returns
index_time,,,,,,,,,,,,,,,,,,,,,
2016-12-16,2016-12-16 00:00:00.000000,24.41,24.73,23.94,23.980,483544,-0.023218,2016-12-16 00:00:00.000000,242.80,243.19,...,39.32,7298861,-0.005564,2016-12-16 00:00:00.000000,14.29,14.47,14.2300,14.375,4516341,0.017339
2016-12-19,2016-12-19 00:00:00.000000,24.00,24.01,23.55,23.790,288149,-0.007923,2016-12-19 00:00:00.000000,238.34,239.74,...,39.45,3436478,0.003306,2016-12-19 00:00:00.000000,14.34,14.60,14.3000,14.360,3944657,-0.001043
2016-12-20,2016-12-20 00:00:00.000000,23.75,23.94,23.58,23.820,220341,0.001261,2016-12-20 00:00:00.000000,240.52,243.65,...,39.74,2940991,0.007351,2016-12-20 00:00:00.000000,14.73,14.82,14.4100,14.490,5207412,0.009053
2016-12-21,2016-12-21 00:00:00.000000,23.90,23.97,23.69,23.860,249189,0.001679,2016-12-21 00:00:00.000000,242.24,242.40,...,40.09,5826704,0.008807,2016-12-21 00:00:00.000000,14.45,14.54,14.2701,14.380,3901738,-0.007591
2016-12-22,2016-12-22 00:00:00.000000,23.90,24.01,23.70,24.005,383139,0.006077,2016-12-22 00:00:00.000000,241.23,242.86,...,39.68,4338385,-0.010227,2016-12-22 00:00:00.000000,14.33,14.34,13.9301,14.040,3874004,-0.023644


### Step 3.2: Calculating the Average Daily Returns of the ETF Portfolio

In this step, we compute the **average daily returns** of the ETF portfolio, which is composed of four equally weighted assets. By averaging the individual daily returns of these assets, we derive the daily performance of the overall portfolio. This serves as a foundation for further performance metrics, such as **annualized returns** and **cumulative returns**.

The approach assumes that each asset contributes equally to the portfolio’s overall performance, which is standard for an equally weighted ETF. This method provides an aggregate measure of the portfolio's daily movement, smoothing out the volatility of individual assets while reflecting the overall trend.

The calculation is implemented by applying the `.mean()` function across the daily returns of all assets, averaging along the row axis (`axis=1`), to generate a time series of portfolio returns:

```python
etf_portfolio_returns = etf_portfolio['daily_returns'].mean(axis=1)
```

This output will be used in subsequent calculations to assess the ETF's long-term performance and risk profile.

In [22]:
# Create a DataFrame that averages the “daily_returns” columns for all four assets.
etf_portfolio_returns = etf_portfolio['daily_returns'].mean(axis=1)

print('\033[1mETF Portfolio Returns (%)')
display(round((etf_portfolio_returns * 100), 2).head(10))

ETF Portfolio Returns (%)


index_time
2016-12-16   -0.70
2016-12-19   -0.12
2016-12-20    0.86
2016-12-21   -0.10
2016-12-22   -0.82
2016-12-23   -0.12
2016-12-27    0.03
2016-12-28   -0.42
2016-12-29   -0.51
2016-12-30   -0.37
dtype: float64

In [23]:
# As a second view, we create a DataFrame that displays the value of the “daily_returns” for all four assets only, and assign an index.
query2 = """
    SELECT  
        GDOT.time
        , GDOT.daily_returns as 'GDOT.daily_returns'
        , GS.daily_returns as 'GS.daily_returns'
        , PYPL.daily_returns as 'PYPL.daily_returns'
        , SQ.daily_returns as 'SQ.daily_returns'
    FROM GDOT, GS, PYPL, SQ
    WHERE GDOT.time = GS.time
        AND PYPL.time = SQ.time
        AND GDOT.time = PYPL.time;
"""

# Read the query data from the database into a Pandas DataFrame
fmt = '%Y%m%d %H:%M:%S'
etf_portfolio2 = pd.read_sql_query(
    query2,
    engine,
    parse_dates={'time': fmt}
)
etf_portfolio2 = etf_portfolio2.set_index("time") 

In [24]:
print('\n')
print('\033[1m                         Daily individual returns in time (%)')
display(round((etf_portfolio2 * 100), 2))

# We repeat the calculation of average daily returns using this table
etf_portfolio_returns = etf_portfolio2.mean(axis=1)

# Review the resulting DataFrame
display("ETF Returns (%)")
display(round(etf_portfolio_returns * 100, 2))



                         Daily individual returns in time (%)


,GDOT.daily_returns,GS.daily_returns,PYPL.daily_returns,SQ.daily_returns
time,,,,
2016-12-16,-2.32,-1.67,-0.56,1.73
2016-12-19,-0.79,0.08,0.33,-0.10
2016-12-20,0.13,1.66,0.74,0.91
2016-12-21,0.17,-0.69,0.88,-0.76
2016-12-22,0.61,-0.52,-1.02,-2.36
...,...,...,...,...
2020-11-30,-4.38,-2.13,1.36,-0.72
2020-12-01,0.45,0.65,1.08,-3.78
2020-12-02,-2.73,2.44,-1.78,-0.44


'ETF Returns (%)'

time
2016-12-16   -0.70
2016-12-19   -0.12
2016-12-20    0.86
2016-12-21   -0.10
2016-12-22   -0.82
              ... 
2020-11-30   -1.46
2020-12-01   -0.40
2020-12-02   -0.63
2020-12-03    1.12
2020-12-04    0.91
Length: 999, dtype: float64

### Step 3.3: Use the average daily returns in the etf_portfolio_returns DataFrame to calculate the annualized returns for the ETF.

In [25]:
# Use the average daily returns over 252 days in the etf_portfolio_returns DataFrame 
# to calculate the annualized return for the portfolio. 
annualized_etf_portfolio_returns = etf_portfolio_returns.mean() * 252

print(f"The expected annualized return, calculated using daily average return in the period, times 252 trading days is: {annualized_etf_portfolio_returns * 100:, .2f}% ")



The expected annualized return, calculated using daily average return in the period, times 252 trading days is: 43.83% 


### Step 3.4. We use the average daily returns in the `etf_portfolio_returns` DataFrame to calculate the cumulative returns of the ETF portfolio. With that, we calculate performance.


In [29]:
# Calculate the cumulative returns (growth of 1[USD] initial investment)
etf_cumulative_returns = (1 + etf_portfolio_returns).cumprod()
etf_cumulative_returns

time
2016-12-16    0.992962
2016-12-19    0.991755
2016-12-20    1.000251
2016-12-21    0.999246
2016-12-22    0.991010
                ...   
2020-11-30    4.374534
2020-12-01    4.357078
2020-12-02    4.329679
2020-12-03    4.378371
2020-12-04    4.418250
Length: 999, dtype: float64

In [30]:
# 
growth_of_1usd_initial_investment = etf_cumulative_return_above_initial_investment + 1

# Display the final cumulative return value
print(f"The cumulative return of the investment in the full period, above the initial investment (no-annualized) is of {etf_cumulative_return_above_initial_investment*100:,.2f}%")
print(f"The growth of $1.00 initial investment in the full period is ${growth_of_1usd_initial_investment:.2f} ")

The cumulative return of the investment in the full period, above the initial investment (no-annualized) is of 341.83%
The growth of $1.00 initial investment in the full period is $4.42 


In [31]:
# Adjusting columns names for proper graph variables
etf_cumulative_returns_df = pd.DataFrame(etf_cumulative_returns, columns=['Growth of 1[USD] Initial Investment'])

etf_cumulative_returns_df.tail()

,Growth of 1[USD] Initial Investment
time,
2020-11-30,4.374534
2020-12-01,4.357078
2020-12-02,4.329679
2020-12-03,4.378371
2020-12-04,4.418250


### Step 3.5: Using hvPlot, we create an interactive line plot that visualizes the cumulative return values of the ETF portfolio.

In [32]:
# Using hvplot, create an interactive line plot that visualizes the ETF portfolios cumulative return values.
etf_cumulative_returns_df.hvplot(
    title=f"ETF - Equally Weighted FinTech Stocks (GDOT, GS, PYPL, SQ) Growth of 1 USD Initial Investment -- {start.date()} to {end.date()}",
    ylabel="Cumulative Investment [$]",
    xlabel="Date",
    width=1100,
)

:Curve   [time]   (Growth of 1[USD] Initial Investment)

### 4. Save the ETF returns in a new table in a new database. This database will contain ETF returns exclusively.
We want to keep the original etf.db file intact for future use of the script.

In [37]:
# Saving the results on a new database called etf_returns.
# Create a temporary SQLite database and save it as etf_returns.db
database_connection_string2 = 'sqlite:///etf_returns.db'

# Create an engine to interact with the SQLite database
engine2 = sqlalchemy.create_engine(database_connection_string2)
inspect(engine2).get_table_names()

['ETF_returns']

In [38]:
# We prepare column names for the data
etf_portfolio_returns_df = etf_portfolio_returns.to_frame()
etf_portfolio_returns_df.columns = ['Equally_weighted']
etf_portfolio_returns_df

,Equally_weighted
time,
2016-12-16,-0.007038
2016-12-19,-0.001216
2016-12-20,0.008567
2016-12-21,-0.001004
2016-12-22,-0.008243
...,...
2020-11-30,-0.014635
2020-12-01,-0.003990
2020-12-02,-0.006288


In [39]:
# We insert in the new table the ETF equally weighted returns just calculated
etf_portfolio_returns_df.to_sql(
    'ETF_returns', engine2, index=True, if_exists='replace'
)

In [36]:
# We inspect the column names in the table to confirm is there 
columns_table = inspect(engine2).get_columns('ETF_returns')

# We print them with the datatype
for c in columns_table:
   print(c['name'], c['type'])

time DATETIME
Equally_weighted FLOAT


In [74]:
# We check the data have been saved in etf_returns.db
read_data_query = """
    SELECT * from ETF_returns
"""
pd.read_sql_query(read_data_query, con=engine2)

,time,Equally_weighted
0,2016-12-16 00:00:00.000000,-0.007038
1,2016-12-19 00:00:00.000000,-0.001216
2,2016-12-20 00:00:00.000000,0.008567
3,2016-12-21 00:00:00.000000,-0.001004
4,2016-12-22 00:00:00.000000,-0.008243
...,...,...
994,2020-11-30 00:00:00.000000,-0.014635
995,2020-12-01 00:00:00.000000,-0.003990
996,2020-12-02 00:00:00.000000,-0.006288
997,2020-12-03 00:00:00.000000,0.011246


#### Deployment of the Notebook as a Web Application

For this part, we use the Voilà library to deploy the notebook as a web application locally on the computer.
See the [video here](https://www.youtube.com/watch?v=wyaDnec7fGk) 
